[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jabascal/deep-learning-for-computer-vision-with-keras/blob/main/image_classification_using_transfer_learning.ipynb)

# Image classification using transfer learning

The objectives of this notebook are to create a classifier that leads to the largest accuracy on the given dataset using a transfer learning and fine-tuning approach. 

For *transfer learning*, we use the pretrained weights of an architecture pretrained on a large-scale dataset (ImageNet), excluding the top classification layer. Then, we freeze the rest of layers and add a classification layer adapted to the number of classes in our specific dataset. Finally, we train the new model, which has a very low number of weights (few thousands) compared to  the base model (with several millions), on a specific and smaller dataset. 

For *fine tuning*, we unfreeze few or all of the top layers of the base model and then tune the weights to obtain higher accuracy. 

## Import dependencies

In [ ]:
mode_install = False
if mode_install:
    !pip install tensorflow \
        pillow \
        matplotlib \
        pandas \
        scikit-learn

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
print(f"TF version {tf.__version__}")

import pathlib
import random
import time
import PIL

To access **GPU** go to 'Runtime/Change runtime type' and to check if GPU is available and resources, run the following code:

In [ ]:
# Device name
tf.test.gpu_device_name()

# GPU (Tesla), memory limit (14GB)
from tensorflow.python.client import device_lib
device_lib.list_local_devices()


In [ ]:
# Memory resources
!cat /proc/meminfo

## Download data

### Mount google drive and paths

Start by mounting your google drive:   

In [ ]:
# Mount google drive to access files via colab
mode_colab = True
if mode_colab:
    from google.colab import drive
    drive.mount("/content/gdrive")

Specify the path for saving data and results.

In [ ]:
# Path notebook
#notebook_dir = "/content/gdrive/MyDrive/Colab_Notebooks/deep-learning-for-computer-vision-with-keras/"
#os.chdir(notebook_dir)

name_save = "flower"

# Path results
if mode_colab:
      save_dir = "/content/gdrive/MyDrive/Colab_Notebooks/Results/flower_photos"
      data_dir = '/content/gdrive/MyDrive/Colab_Notebooks/Data'
else:
      save_dir = "../../Results/flower_photos"
      data_dir = '../../Data'
if os.path.exists(save_dir) is False:
      os.mkdir(save_dir)
      print(f"Directory: {save_dir} created.")
if os.path.exists(data_dir) is False:
      os.mkdir(data_dir)
      print(f"Directory: {data_dir} created.")

### Download data

Download the data subset automatically into your drive to the directory *data_dir*: '/content/gdrive/MyDrive/Colab_Notebooks/Data/'

In [ ]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True, cache_dir=data_dir)
print(f"Data downloaded to {os.path.abspath(data_dir)}")
!rm "{data_dir}/../flower_photos.tar.gz"
#os.chdir(data_new_dir)
#!wget -P path_notebook -O flower_photos.tgz https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz
#!tar -xzf os.path.join(data_dir, 'flower_photos.tgz')
#!rm flower_photos.tgz

## Define parameters and general functions

In [ ]:
# Image sizes (depends on the model)
img_height = 180 
img_width = 180

batch_size = 256
learning_rate = 1e-4

## Data pipeline

### Create dataset

We use the tensorflow data API to automatize the data pipeline, chaining transformations (preprocessing and data augmentation), shuffling data. 

Next, we create dataset using 'image_dataset_from_directory' to get similar labeled dataset objects to specified folders. Split data into train, validation and test. 

In [ ]:
# Create data set 
# Split in training and validation
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  shuffle=True)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  shuffle=True)

# Class names
class_names = train_ds.class_names
num_classes = len(class_names)
print(f'Classes names: {class_names}')

# Split Validation and Test
val_batch_size = val_ds.cardinality().numpy()
test_ds = val_ds.take(int(0.5*val_batch_size))
val_ds = val_ds.skip(int(0.5*val_batch_size))

An efficient pipeline can be obtained using 'cache' which keeps the data in RAM memory after the first epoch and 'prefetch' which allows to prepare data for next batch while the model is being trained for the current batch on the GPU. Data is shuffled at each iteration for training data.

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

# shuffle after cache
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

### Data visualization 

In [ ]:
# Display several images
fig = plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")
    fig.savefig(os.path.join(save_dir, f"{name_save}_grid.png"), bbox_inches='tight', dpi=300)

Display an image per class

In [ ]:
# Display an image per class
fig, axs = plt.subplots(1, num_classes, figsize=(15, 5))
for i, class_name in enumerate(class_names):
    # Get an image from the class
    image_path = os.path.join(data_dir, class_name, os.listdir(os.path.join(data_dir, class_name))[0])
    image = PIL.Image.open(image_path)
    
    # Display the image
    axs[i].imshow(image)
    axs[i].set_title(class_name)
    axs[i].axis('off')

plt.show()
fig.savefig(os.path.join(save_dir, f"{name_save}_classes.png"), bbox_inches='tight', dpi=300)


Images contain several objects and different background, which may harden the classification task.

Number of data per class:

In [ ]:
# Number of samples per class
data_dir = pathlib.Path(data_dir)

class_counts = []
for class_name in class_names:
    class_count = len(list(data_dir.glob(class_name+'/*.jpg')))
    class_counts.append(class_count)

fig = plt.figure(figsize=(4,4))
plt.barh(class_names, class_counts)
plt.title('Number per class')
plt.show()
fig.savefig(os.path.join(save_dir, f"{name_save}_classes_counts.png"), bbox_inches='tight', dpi=300)

### Data augmentation

Data augmentation is performed using random flip,  rotation and zooming. Many more operataions can be used, such as random contrast, brightness, hue, saturation, etc. See [tf.image](https://www.tensorflow.org/api_docs/python/tf/image/) for more details.

In [ ]:
# Data augmentation
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal", input_shape=(img_height, img_width, 3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ], name="data_augmentation"
)

# Data augmentation example
fig = plt.figure(figsize=(7, 7))
for images, _ in train_ds.take(1):
  for i in range(9):
    augmented_images = data_augmentation(images)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[0].numpy().astype("uint8"))
    plt.axis("off")
fig.savefig(os.path.join(save_dir, f"{name_save}_augmentation.png"), bbox_inches='tight', dpi=300)

## Model, training and assessment: A transfer learning approach

### Create the model

We use transfer learning using a pretrained model that has been trained on a very large dataset (ImageNet). We can try different models: 'Xception' which provides a high top-5 accuracy (with 20 M parameters) and 'MobileNetV2' which provides great accuracy for a relatively small model size (4 M parameters). 

We load the model but skip the 'top' layer to tailored our model to the classes in the dataset. Then, we freeze their layers to train on a small dataset. We also define the model and specify their preprocessing steps. 

Instead of creating and training the model, you can load the trained model: run the code below.

In [ ]:
model_load_model = False
if model_load_model is True: 
  model = keras.models.load_model(os.path.join(save_dir,'flower_photos_Xception_TF62it.h5'))
  epochs = 62

In [ ]:
IMG_SHAPE = (img_height, img_width) + (3,)

# Download pretrained base model
base_model_name = 'mobilenet_v2' # mobilenet_v2, Xception, vgg16
if base_model_name == 'mobilenet_v2':
    # mobilenet_v2: small networks
    # Param M: 4.24, top-1 acc: 70.9, top-5 acc:	89.9
    # Imagenet ILSVRC-2012-CLS 
    # Timing: # GTX: 7s and 3s, FT: 
    base_model = tf.keras.applications.MobileNetV2(
            input_shape=IMG_SHAPE,                                                   
            include_top=False,                                                   
            weights='imagenet')
    preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input    
elif base_model_name == 'Xception':
    # Xception
    # 20 M parameters
    # no smaller than 71. E.g. (150, 150, 3) 
    # Timing: # GTX: 10s and 6s, FT: 15s and 9s
    base_model = keras.applications.Xception(
            input_shape=IMG_SHAPE,                                                   
            include_top=False,                                                   
            weights='imagenet')
    preprocess_input = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)

# Freeze weights
base_model.trainable = False

base_model.summary()

To create the model, we define the top layers, a pooling layer and the classification layer, specifying the number of classes, and then concatenate all layers. Data augmentation and pre-processing are included as bottom layers in the model. 

We also add a dropout layer to avoid overfitting. It drops a fraction of the input units and has a similar effect as L2 regularization.

In [ ]:
def get_model(inputs_shape, num_classes=num_classes, dropout_rate=0.2):

    # Average pooling layer to pass from block 6x6x1280 to vector 1x1280
    global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

    # Multiclass classification layer
    prediction_layer = tf.keras.layers.Dense(num_classes)

    # Create the model
    inputs = tf.keras.Input(shape=inputs_shape)
    x = data_augmentation(inputs)
    x = preprocess_input(x)        
    x = base_model(x, training=False)
    x = global_average_layer(x)
    x = layers.Dropout(dropout_rate)(x)
    outputs = prediction_layer(x)
    model = tf.keras.Model(inputs, outputs)

    model.summary()
    return model

model = get_model(inputs_shape=(img_height, img_width, 3), 
                  num_classes=num_classes)

The pretrained Xception model has 21 M parameters, which have been pretrained on 14 M images. On the transfer learning step we aim to fit instead only 10 thousand parameters using 4 thousand images. 

We define callbacks to for tensorboard (weights analysis and profiler) and for early stopping. For other callbacks, see [tf.keras.callbacks](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/).

In [ ]:
# LOG DIRECTORY FOR TENSORBOARD
def get_run_logdir(path_log):
    now = time.strftime("run_%Y-%m-%d-%H-%M-%S")
    log_dir = os.path.join(save_dir, "runs", now)
    print(f"Log directory: {log_dir}")
    return log_dir

# CALLBACKS 
# Tensorboard
# tensorboard --logdir=log_dir
log_dir = get_run_logdir(save_dir)    

# Enable visualizations for Tensorboard
tensorboard_cb = keras.callbacks.TensorBoard(
    log_dir,
    histogram_freq=10, # Histograms of weights
    write_images=True, # Images for weights
    profile_batch = '2,3') # Profiling: improve performance https://www.tensorflow.org/guide/profiler

# Early stopping based on a given metric
earlystop_cb = keras.callbacks.EarlyStopping(
    patience=20, 
    monitor='val_accuracy',
    restore_best_weights=True)

# Model checkpoint
tf_vs = str(tf.__version__).replace(".", "-")
checkpoint_path = os.path.join(save_dir, "training",
                               f"{name_save}_{base_model_name}_TF{tf_vs}"+"cp-{epoch:04d}.ckpt")
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_path,
    save_weights_only=True,
    save_best_only=True,
    verbose=1,
    save_freq=10)

callbacks = [tensorboard_cb, earlystop_cb]

Checkpoints save only weights, in binary format on one or more shards, not the model. Weights are save in `.ckpt` format. To save in  HDF5 format add the extension `.h5` to the file name. 

To load the model, you need to define the model and then load the weights.
```
    # Save (manually) the weights only 
    model.save_weights(checkpoint_path)

    # Create a new model instance
    model = create_model()

    # Load the weights
    model.load_weights(checkpoint_path)
```


### Train the model 

We compile the model, specifying the loss and learning rate, and train. We aim to to obtain a high accuracy as we are fine tuning lower layers in a later fine tuning step. 

In [ ]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train a large number of steps with early stopping criterion
epochs=70
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
  callbacks=callbacks
)


Training took $10$ s per epoch (took $6$ s/step on GTX 1080 Ti) and used 2.5 GB. 

Below, we display loss and accuracy. We see that the validation loss is starting to flatten out, with a final accuracy of $89$ %. We could take more steps but we leave it for the fine tuning step. 

We remark that even though we use a early stopping criterion, the model did not stop (it run only $63$ epochs on my local computer); more iterations could be set. 

In [ ]:
# Display loss and accuracy
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

# Visualize metrics
epochs_range = range(len(acc))

fig = plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
fig.savefig(os.path.join(save_dir, f"{name_save}_accuracy.png"), bbox_inches='tight', dpi=300)
#
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
fig.savefig(os.path.join(save_dir, f"{name_save}_loss.png"), bbox_inches='tight', dpi=300)
plt.show()

print('Accuracy on validation set at the final step %.2f' %(acc[-1]))

Save the entired trained model we use `tf.keras.Model.save` to save a model's architecture, weights and training configuration in a single file. Threre are several formats to save the model: `.keras` (Keras v3) and two legacy formats `.h5` and `SavedModel`, specifying `save_format`. These can be load in web browser or convert them to TFLite format for mobile devices.

In [ ]:
# Save the model 
model_path = os.path.join(save_dir, f'{name_save}_{base_model_name}_TF{tf_vs}_ep{str(history.epoch[-1])}.keras')
model.save(model_path)
print(f"Model saved in {os.path.join(save_dir, name_save)}")
print(os.listdir(save_dir))

Save history.

In [ ]:
# Save history
import json
with open(os.path.join(save_dir, f'{name_save}_hist.json'), 'w') as file:
  json.dump(history.history, file)

We asses the model on the test dataset using `Model.evaluate`, which returns the loss and metrics values.

In [ ]:
loss_assess, accuracy_assess = model.evaluate(test_ds)
print('Test accuracy :', accuracy_assess)

## Model, training and assessment: Further fine tuning step

To improve further the accuracy we fine tune lower layers. Once the model has converged on the new data, we unfreeze part of the base model and retrain the whole model end-to-end with a very low learning rate. The 'Xception' model has $132$ layers. We fine tune all layers after layer $70$, which leads to $10$ M from the $20$ M parameters in the full model. This needs $17$ s per epoch.

Instead of creating and training the model, you can load the trained model: run the code below.

In [ ]:
model_load_model = False
if model_load_model is True: 
  model = keras.models.load_model(os.path.join(save_dir,'flower_photos_Xception_TF62it_FTat70_it105.h5'))

In [ ]:
# FINE TUNING
# Unfreeze the base model
base_model.trainable = True

# Number layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 70

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False
    
# Recompile your model after you make any changes
model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate/10),  # Lower learning rate
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

Train the new model.

In [ ]:
# Train (continue training)
fine_tune_epochs = 70
total_epochs = epochs + fine_tune_epochs    
history_fine = model.fit(train_ds,
                          epochs=total_epochs,
                          initial_epoch=history.epoch[-1],
                          validation_data=val_ds,
                          callbacks=callbacks)


In [ ]:
# Display losses
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

epochs_range = range(len(acc))
fig = plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.ylim([0.5, 1])
plt.plot([epochs-1,epochs-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
#
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.ylim([0, 1.0])
plt.plot([epochs-1,epochs-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()
fig.savefig(os.path.join(save_dir, f"{name_save}_loss_fine.png"), bbox_inches='tight', dpi=300)

Predict and get accuracy on test set of $95$ %! There is  a clear jump in accuracy after fine tuning. We remark that the metric used for early stoping is the 'accuracy' so although the validation loss is decreasing after around $85$ epochs, the accuracy flattens and oscillates, stopping at $106$ epochs. Setting the epochs to $85$ may lead to a model that generalizes better.  

### Model assessment

Load the saved model. 

In [ ]:
# model = tf.keras.models.load_model(model_path)

We asses the model on the test dataset using `Model.evaluate`, which returns the loss and metrics values.

In [ ]:
loss_assess, accuracy_assess = model.evaluate(test_ds)
print('Test accuracy :', accuracy_assess)

Alternatively, we can load the data, predict and compute the desired metric.

In [ ]:
# Load test data from test dataset. 
def get_imgs_from_dataset(ds_test, ds_test_size):
    # Take images from data set ds_test: (data_test, data_test_noisy) 
    data_test = []
    label_test = []
    count = 0
    for img, label in ds_test.take(ds_test_size):
        data_test_this = img.numpy()  
        label_test_this = label.numpy() 
        if count == 0:
            data_test = data_test_this
            label_test = label_test_this
            count = 1
        else:            
            data_test = np.append(data_test, data_test_this, axis=0)
            label_test = np.append(label_test, label_test_this, axis=0)
    return data_test, label_test
    
# Test data
data_test, label_test = get_imgs_from_dataset(val_ds, len(val_ds)-1)

Predict and get accuracy on test set of $88$ %. This accuracy depends on the number of epochs, which can vary as we are using early stopping criterion. Higher number of epochs would improve accuracy, but we leave it for the fine-tuning step.

In [ ]:
# Predict 
data_pred = model.predict(data_test)
data_pred_class = np.argmax(data_pred, axis=1)

acc_fn = tf.keras.metrics.Accuracy()
test_acc = acc_fn(data_pred_class, label_test)
print('Accuracy for %s with fine tuning is %.2f' % (base_model_name, test_acc))

Save the trained model. 

In [ ]:
model_name = f'{name_save}_{base_model_name}_TF{tf_vs}_ep{history.epoch[-1]}_FTat{fine_tune_at}_ep{history_fine.epoch[-1]}'
model.save(os.path.join(save_dir, f'{model_name}.keras'))
print(f"Model saved in {os.path.join(save_dir, name_save)}")
print(os.listdir(save_dir))

Save history.

In [ ]:
# Save history
with open(os.path.join(save_dir, f'{model_name}_history.json'), 'w') as file:
  json.dump(history_fine.history, file)

## Final analysis and image exploration

### Confusion matrix and display misclassifications

To analyze the final results we build the confusion matrix. 

In [ ]:
# Confusion matrix
from sklearn.metrics import confusion_matrix
import pandas as pd

def confusion_matrix_analysis(label_test, data_pred_class, class_names):
    confusion_mat = confusion_matrix(label_test, data_pred_class)
    confusion_df = pd.DataFrame(confusion_mat, columns=class_names, index=class_names)
    print('Confusion matrix (row: true, columns: predictions):')
    print(confusion_df)
    # Largest errors: roses as tulips, daisy as tulips, dandelion as tulips
    
    # Confusion matrix off-diagonals
    confusion_mat_off = np.copy(confusion_mat)
    np.fill_diagonal(confusion_mat_off, 0)
    confusion_as = np.sum(confusion_mat_off, axis=0)
    confusion_as_df = pd.Series(confusion_as, index=class_names)
    print('')
    print('Missclassified as ...')
    print(confusion_as_df)
    # flowers are misclassified mostly as tulips, 
    # so tulips images must be revisited 
    # dandelion presents the lowest (most data)
    
    confusion_for = np.sum(confusion_mat_off, axis=1)
    confusion_for_df = pd.Series(confusion_for, index=class_names)
    print('')
    print('Missclassifications for ...')
    print(confusion_for_df)
    # while tulips are the flower with lowest number of errors
    return confusion_df, confusion_as_df, confusion_for_df

confusion_df, confusion_as_df, confusion_for_df = confusion_matrix_analysis(label_test, data_pred_class, class_names)


In [ ]:
def plot_confusion_matrix(cm, class_names):
    """
    Returns a matplotlib figure containing the plotted confusion matrix.

    Args:
        cm (array, shape = [n, n]): a confusion matrix of integer classes
        class_names (array, shape = [n]): String names of the integer classes
    """

    import itertools

    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)

    # Compute the labels from the normalized confusion matrix.
    labels = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

    # Use white text if squares are dark; otherwise black.
    threshold = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = "white" if cm[i, j] > threshold else "black"
        plt.text(j, i, labels[i, j], horizontalalignment="center", color=color)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return figure

fig = plot_confusion_matrix(confusion_df.to_numpy(), class_names)
fig.savefig(os.path.join(save_dir, f"{model_name}_confmat.png"), bbox_inches='tight', dpi=300)

We get almost perfect classification for *tulips*. *Daisies* and *roses* are also well classified. *Sunflowers* and *dandelions* have the largest errors, which should be investigated. 

We display the misclassified results for analysis.

In [ ]:
# Explore misclassified results 
def display_misclassified_images(data_test, label_test, data_pred_class, class_names, misclassified_only=True):
    test_misclassified = [i for i in range(len(label_test)) if (data_pred_class[i] != label_test[i])]
    classes_misclassified = []
    for class_ind, class_name in enumerate(class_names):
        label_test[test_misclassified] == class_ind
        class_misclassified = [i for i in range(len(label_test)) 
            if (data_pred_class[i] != label_test[i])
            and (label_test[i] == class_ind)]
        classes_misclassified.append(class_misclassified)
        
        # Display misclassified images for each class
        print('Misclassified images for ' + class_name)
        plt.figure(figsize=(15, 15))
        for i in range(min(20,len(class_misclassified))):
            ax = plt.subplot(5, 4, i + 1)
            data_test_ind = class_misclassified[i]
            plt.imshow(data_test[data_test_ind].astype("uint8"))
            plt.title(class_names[label_test[data_test_ind]] + ' as ' + class_names[data_pred_class[data_test_ind]],
                      fontsize=18)
            plt.axis("off")
        plt.show()
        
        if misclassified_only is False:
            # Display well classified results for comparison
            class_well_classified = [i for i in range(len(label_test)) 
                if (data_pred_class[i] == label_test[i])
                and (label_test[i] == class_ind)]
            print('Well classified images for ' + class_name)
            plt.figure(figsize=(15, 15))
            for i in range(min(20,len(class_well_classified))):
                ax = plt.subplot(5, 4, i + 1)
                data_test_ind = class_well_classified[i]
                plt.imshow(data_test[data_test_ind].astype("uint8"))
                plt.title(class_names[label_test[data_test_ind]] + ' as ' + class_names[data_pred_class[data_test_ind]],
                          fontsize=18)
                plt.axis("off")
            plt.show()

# Display misclassified results
display_misclassified_images(data_test, label_test, data_pred_class, class_names)

This display raises the following comments for further analysis: 
* **Daisies**: The first image shows that daisies with pink color (not white) can be misclassified. The second image shows that a strong yellow background with reflections hardens the problem. 
This could explained that are misclassified as tuilips. 

* **Dandelions**: Yellow dandelions are misclassified as sunflowers and white dandelions as daisies. 

* **Roses**: The first image is too hard. The others should be well classified (second for the background and the third one because of the zoom).

* **Sunflowers**: Misclassification of sunflowers as roses 
do not make sense; maybe, it is caused of the backgound. 
Looking at the features that are activated may help. 
One of the images does not have flower, which raises the question if the classifier is also learning the leave type for each class.

* **Tulips**: the image seems wrong. We should check the test and training set to remove wrongly labeled images.

Some of these misclassifications changed when re-training the model from scratch or using different base model, which encourages to use ensemble methods for futher accuracy. 

### Asses on new acquired data

We further assess the trained model on new images acquired with a mobile, download to /content/gdrive/MyDrive/Colab_Notebooks/Data 

In [ ]:
# Download new data
#dataset_url = "https://www.dropbox.com/s/sofqjvmlt1omqr6/flowers_mobile.rar?dl=0"
#data_new_dir = tf.keras.utils.get_file('flowers_mobile', origin=dataset_url, extract=True,)
data_new_dir = '/content/gdrive/MyDrive/Colab_Notebooks/Data'
os.chdir(data_new_dir)
!wget -P data_new_dir -O flowers_mobile.rar https://www.dropbox.com/s/sofqjvmlt1omqr6/flowers_mobile.rar?dl=0
!unrar flowers_mobile.rar -idq
!rm flowers_mobile.rar
data_new_dir = os.path.join(data_new_dir, 'flowers_mobile')

print(f"Dataset saved at: {data_new_dir}")

Define dataset and load images

In [ ]:
# New test data
test_new_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_new_dir,
  validation_split=None,
  image_size=(img_height, img_width),
  batch_size=batch_size)

data_test_new, label_test_new = get_imgs_from_dataset(test_new_ds, len(test_new_ds))

Classify new photos acquire with the mobile, even though images have been loaded upside down. Most images are well classified, including daisies out of focus. Photos with many daisies on green background are misclassied. 

In [ ]:
# Predict 
data_pred_new = model.predict(data_test_new)
data_pred_new_class = np.argmax(data_pred_new, axis=1)

acc_fn = tf.keras.metrics.Accuracy()
test_new_acc = acc_fn(data_pred_new_class, label_test_new)
print('Accuracy on new data is %.2f' % (test_new_acc))

In [ ]:
# Display misclassified images
display_misclassified_images(data_test_new, label_test_new, data_pred_new_class, class_names,misclassified_only=False)

### Conclusions and further analysis

We have obtained an accurate classifier that reached $95$ % accuracy on a relatievely small data set using the 'Xception' model (we obtain $93$ % using the 'MobileNetV2' model using the same procedure; model saved in 'Results' folder). Training with VGG16, VGG19 and . Furter analysis should be considered for final accuracy quantification, maybe using generalized cross validation. 

Further work should focus on pursuing the misclassifications:
* From the results obtained, ensemble methods may lead to futher accuracy, as different models misclassified different images. 

* Images that were consistently misclassified, such as daisies, were due to color resembles to other classes, which could be improved. 

* Misclassifications due to zooms and backgrounds could be investigate by looking at activations of features. 

* Cleaning the data set and doing GCV could also help due the size of the dataset. 

* Working on the data rather than on the model seem to be a current approach to further improve results on the community. 

* It would be also nice to visualize the data using an embedding (eg. embedding projector on TF).

An error table that includes the different possible sources of errors and their ocurrence should provide an ordered list of priorities. 

 